In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime, pickle, os
from glob import glob
from tqdm.auto import tqdm
%matplotlib inline

import torch
from torch.utils.data import Dataset as _Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch import nn

from IPython.display import display, display_markdown

In [2]:
headline_features = pd.read_pickle('../data/headline_features.pkl')
new_headline_features = pd.read_pickle('../data/new_headline_features.pkl')
new_headline_features.index = new_headline_features.index.tz_convert(headline_features.index.tz)

# 結合
headline_features = pd.concat([headline_features, new_headline_features], axis=0).copy()

# 確認する。
display(headline_features.head(3))

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
publish_datetime,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:00:00+09:00,-0.440823,0.191443,-0.008909,-0.319594,-0.072300,0.466741,-0.274743,0.445048,0.498942,0.032077,...,-0.343381,0.090035,0.199363,-0.338375,0.670792,0.026400,0.632040,-0.433479,-0.011136,-0.051264
2020-01-01 00:00:00+09:00,-0.351773,-0.027478,-0.060213,-0.512602,0.254371,0.128152,0.160517,0.154172,0.151672,0.255984,...,-0.078464,0.179369,0.076657,-0.030442,-0.168868,-0.192623,0.602098,-0.076874,-0.616428,-0.406168
2020-01-01 00:00:00+09:00,-0.115327,0.017725,-0.129011,-0.553259,0.096930,0.092610,0.150430,-0.043717,-0.105239,-0.155988,...,-0.505299,0.315577,0.302115,0.032662,0.129836,-0.134002,0.500410,-0.120089,-0.640578,-0.581745


In [3]:
# stock_priceとstock_listをロードします。
stock_price = pd.read_csv('../data/stock_price.csv.gz')
stock_list = pd.read_csv('../data/stock_list.csv.gz')
new_stock_price = pd.read_csv('../data/new_stock_price_all.csv', index_col=[0])

# 結合する
stock_price = pd.concat([stock_price, new_stock_price], axis=0).copy()
stock_price.sort_values(by=['Local Code', 'EndOfDayQuote Date'], inplace=True)
stock_price = pd.merge(stock_price, stock_list[['Local Code', '17 Sector(Code)']], on='Local Code') # セクター番号を追加
stock_price.reset_index(drop=True, inplace=True)
display(stock_price.head())

,Local Code,EndOfDayQuote Date,EndOfDayQuote Open,EndOfDayQuote High,EndOfDayQuote Low,EndOfDayQuote Close,EndOfDayQuote ExchangeOfficialClose,EndOfDayQuote Volume,EndOfDayQuote CumulativeAdjustmentFactor,EndOfDayQuote PreviousClose,EndOfDayQuote PreviousCloseDate,EndOfDayQuote PreviousExchangeOfficialClose,EndOfDayQuote PreviousExchangeOfficialCloseDate,EndOfDayQuote ChangeFromPreviousClose,EndOfDayQuote PercentChangeFromPreviousClose,EndOfDayQuote VWAP,17 Sector(Code)
0,1301,2016/01/04,2800.0,2820.0,2740.0,2750.0,2750.0,32000.0,0.1,2770.0,2015/12/30,2770.0,2015/12/30,-20.0,-0.722,2778.250,1
1,1301,2016/01/05,2750.0,2780.0,2750.0,2760.0,2760.0,20100.0,0.1,2750.0,2016/01/04,2750.0,2016/01/04,10.0,0.364,2761.990,1
2,1301,2016/01/06,2760.0,2770.0,2740.0,2760.0,2760.0,15000.0,0.1,2760.0,2016/01/05,2760.0,2016/01/05,0.0,0.000,2758.867,1
3,1301,2016/01/07,2740.0,2760.0,2710.0,2710.0,2710.0,31400.0,0.1,2760.0,2016/01/06,2760.0,2016/01/06,-50.0,-1.812,2733.471,1
4,1301,2016/01/08,2700.0,2740.0,2690.0,2700.0,2700.0,26200.0,0.1,2710.0,2016/01/07,2710.0,2016/01/07,-10.0,-0.369,2709.122,1


In [4]:
# stock_listから投資対象銘柄を取得し、stock_priceの銘柄を絞り込む
codes = stock_list[stock_list["universe_comp2"] == True]["Local Code"].values
stock_price = stock_price.loc[stock_price.loc[:, "Local Code"].isin(codes)]
stock_price = stock_price[['EndOfDayQuote Date', 'Local Code', "EndOfDayQuote Open", 
                           "EndOfDayQuote ExchangeOfficialClose", "17 Sector(Code)"]]

# それぞれのcolumn名をわかりやすく変更する
stock_price = stock_price.rename(columns={
    'EndOfDayQuote Date': 'date',
    'Local Code': 'asset',
    'EndOfDayQuote Open': 'open',
    'EndOfDayQuote ExchangeOfficialClose': 'close',
    '17 Sector(Code)': 'sector',
})


# データごとにindex形式が異なると大変扱いにくい。下記のコードより特徴量と同様のindexの形式を変更する。
# pd.to_datetimeより、string形式の日付をpd.Timestamp形式に変換する
# pd.Timestamp形式をpd.DatetimeIndex形式に変更し、time zoneをheadline_featuresと同様に設定する。
# この際、headline_featuresとkeywords_featuresはarticlesのindexを使用しているため、timezoneが一致している。どちらを用いても良い。
stock_price['date'] = pd.to_datetime(stock_price['date'])
stock_price['date'] = pd.DatetimeIndex(stock_price['date']).tz_localize(headline_features.index.tz)

In [5]:
dict_stock_price = {} # 0:all_sector, 1~17:sector
all_universe = stock_price.drop('sector', axis=1).set_index(['date', 'asset']).sort_index()
dict_stock_price[0] = all_universe.unstack()['2020-01-01':].copy()

# indexを['sector', date', 'asset']順のpd.MultiIndex形式として設定する。
stock_price = stock_price.set_index(['sector', 'date', 'asset']).sort_index()

for i in range(1, 18):
    # 各セクターの株価データを取り出し、ディクショナリに格納(2020年以降のデータであるので、2020年以前のデータを切り捨てる。)
    dict_stock_price[i] = stock_price.loc[i, :].unstack()['2020-01-01':]
    
# 確認する
display(dict_stock_price[0].head(3))
display(dict_stock_price[1].head(3))

open                                              \
asset                        1301   1332    1333 1375    1377    1379    1407   
date                                                                            
2020-01-06 00:00:00+09:00  2860.0  639.0  2770.0  NaN  3635.0  1956.0  1387.7   
2020-01-07 00:00:00+09:00  2864.0  634.0  2725.0  NaN  3715.0  1965.0  1403.1   
2020-01-08 00:00:00+09:00  2892.0  624.0  2714.0  NaN  3635.0  1943.0  1413.1   

                                                   ...   close          \
asset                        1413    1414    1417  ...    9962    9974   
date                                               ...                   
2020-01-06 00:00:00+09:00  2173.0  4485.0  1642.0  ...  2704.0  5490.0   
2020-01-07 00:00:00+09:00  2130.0  4610.0  1658.0  ...  2765.0  5710.0   
2020-01-08 00:00:00+09:00  2145.0  4555.0  1662.0  ...  2697.0  5790.0   

                                                                            \
asset                        9979     9983    9984    9987    9989    9991   
date                                                                         
2020-01-06 00:00:00+09:00  1663.0  63050.0  4569.0  4335.0  3915.0  1098.0   
2020-01-07 00:00:00+09:00  1685.0  63250.0  4646.0  4430.0  3995.0  1113.0   
2020-01-08 00:00:00+09:00  1680.0  62080.0  4583.0  4400.0  3940.0  1111.0   

                                          
asset                        9994   9997  
date                                      
2020-01-06 00:00:00+09:00  2271.0  703.0  
2020-01-07 00:00:00+09:00  2296.0  706.0  
2020-01-08 00:00:00+09:00  2307.0  689.0  

[3 rows x 3558 columns]

open                                         \
asset                        1301   1332    1333    1377    1379    2001   
date                                                                       
2020-01-06 00:00:00+09:00  2860.0  639.0  2770.0  3635.0  1956.0  1660.0   
2020-01-07 00:00:00+09:00  2864.0  634.0  2725.0  3715.0  1965.0  1665.0   
2020-01-08 00:00:00+09:00  2892.0  624.0  2714.0  3635.0  1943.0  1660.0   

                                                          ...   close          \
asset                        2002    2003    2004   2009  ...    2922    2923   
date                                                      ...                   
2020-01-06 00:00:00+09:00  1896.0  6380.0  3090.0  883.0  ...  1681.0  3665.0   
2020-01-07 00:00:00+09:00  1866.0  6360.0  3130.0  865.0  ...  1725.0  3660.0   
2020-01-08 00:00:00+09:00  1850.0  6350.0  3120.0  871.0  ...  1712.0  3620.0   

                                                                            \
asset                        2925   2929   2930   2931    4526   2296 1375   
date                                                                         
2020-01-06 00:00:00+09:00  2615.0  473.0  708.0  763.0  2002.5  689.0  NaN   
2020-01-07 00:00:00+09:00  2678.0  553.0  736.0  784.0  2037.5  704.0  NaN   
2020-01-08 00:00:00+09:00  2622.0  563.0  729.0  755.0  1992.5  695.0  NaN   

                                
asset                     2932  
date                            
2020-01-06 00:00:00+09:00  NaN  
2020-01-07 00:00:00+09:00  NaN  
2020-01-08 00:00:00+09:00  NaN  

[3 rows x 174 columns]

In [6]:
def _build_weekly_group(df):
    # index情報から、(year, week)の情報を得る。
    return pd.Series(list(zip(df.index.year, df.index.week)), index=df.index)


def build_weekly_features(features, boundary_week):
    assert isinstance(boundary_week, tuple)

    weekly_group = _build_weekly_group(df=features)
    features = features.groupby(weekly_group).apply(lambda x: x[:])

    train_features = features[features.index.get_level_values(0) <= boundary_week]
    test_features = features[features.index.get_level_values(0) > boundary_week]

    return {'train': train_features, 'test': test_features}

def build_weekly_labels(stock_price, boundary_week):
    def _compute_weekly_return(x):
        # その週の初営業日のopenから最終営業日のcloseまでのリターンを計算する。
        weekly_return = ((x['close'].iloc[-1] - x['open'].iloc[0]) / x['open'].iloc[0])

        # その日のvolumneが0であるデータは、openが0となっている。
        # openが0の場合、np.infの値となっているため、np.nanに変換し除去する。
        # 銘柄ごとのリターンを単純平均し、marketのweekly_returnを計算する。
        return weekly_return.replace([np.inf, -np.inf], np.nan).dropna().mean()

    assert isinstance(boundary_week, tuple)

    weekly_group = _build_weekly_group(df=stock_price)
    weekly_fwd_return = stock_price.groupby(weekly_group).apply(_compute_weekly_return).shift(-1).dropna()

    train_labels = weekly_fwd_return[weekly_fwd_return.index <= boundary_week]
    test_labels = weekly_fwd_return[weekly_fwd_return.index > boundary_week]

    train_labels = (train_labels >= 0) * 1.0
    test_labels = (test_labels >= 0) * 1.0

    return {'train': train_labels, 'test': test_labels}

## Pytorch Dataset作成

今回、学習に用いる週次のデータセットにおいて、LSTMの学習には週毎のニュースの件数が若干不足している傾向にあることから、過学習防止のため、少し工夫をしています。具体的には、全体的な特徴量(ニュースの情報)の順序は維持しつつ複数に分割し、その分割の中でシャッフルを行う方法を取ります。この方法を用いることで、モデルに入力するデータを増やすことでき、過学習を防止に繋がる効果が期待できます。

In [7]:
#上記のコードをまとめて、pytorchのDatasetクラスを作成する

class Dataset(_Dataset):
    def __init__(self, weekly_features, weekly_labels, max_sequence_length):
        # 共通する週のみを使うため、共通するindex情報を取得する
        mask_index = (
            weekly_features.index.get_level_values(0).unique() & weekly_labels.index
        )

        # 共通するindexのみのデータだけでreindexを行う。
        self.weekly_features = weekly_features[
            weekly_features.index.get_level_values(0).isin(mask_index)
        ]
        self.weekly_labels = weekly_labels.reindex(mask_index)
        
        # idからweekの情報を取得できるよう、id_to_weekをビルドする
        self.id_to_week = {
            id: week for id, week in enumerate(sorted(weekly_labels.index))
        }

        self.max_sequence_length = max_sequence_length

    def _shuffle_by_local_split(self, x, split_size=50):
        return torch.cat(
            [
                splitted[torch.randperm(splitted.size()[0])]
                for splitted in x.split(split_size, dim=0)
            ],
            dim=0,
        )

    def __len__(self):
        return len(self.weekly_labels)

    def __getitem__(self, id):
        # 付与されたidから週の情報を取得し、その週の情報から、特徴量とラベルを取得する。
        week = self.id_to_week[id]
        x = self.weekly_features.xs(week, axis=0, level=0)[-self.max_sequence_length :]
        y = self.weekly_labels.loc[week]

        # pytorchでは、データをtorch.Tensorタイプとして扱うことが要求される。
        # 全体的な特徴量(ニュースの情報)の順序は維持しつつ、入力とする特徴量を数分割し、その分割の中でシャッフルを行う。
        x = self._shuffle_by_local_split(torch.tensor(x.values, dtype=torch.float))
        y = torch.tensor(y, dtype=torch.float)

        # max_sequence_lengthに最大のsequenceを合わせ、sequenceがmax_sequence_lengthに達しない場合は、前から0を埋め、sequenceを合わせる
        if x.size()[0] < self.max_sequence_length:
            x = F.pad(x, pad=(0, 0, self.max_sequence_length - x.size()[0], 0))

        return x, y

## LSTMによる特徴量合成モデル

In [8]:
class FeatureCombiner(nn.Module):
    def __init__(self, input_size, hidden_size, out_size=18, num_layers=2): # 768, 128
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # LSTMの定義
        # batch_firstより、出力次元の最初がbatchとなる。
        # dropoutを用いて、内部状態のconnectionをdropすることより過学習を防ぐ。
        # Sequenceがかなり長く、入力の始めの方の情報の消失を防ぐため、bidirectionalのモデルを使う。
        self.cell = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.5,
            bidirectional=True,
        )

        # より高次元の特徴量を抽出できるようにするため、classifierの手前で、compress_dim次元への線形圧縮を行う。
        self.compressor = nn.Linear(hidden_size * 2, hidden_size)

        # sentiment probabilityの出力層。
        self.classifier = nn.Linear(hidden_size, out_size)

        # outputの範囲を[0, 1]とする。
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # 入力値xから出力までの流れを定義する。
        output, _ = self.cell(x)
        output = self.sigmoid(self.classifier(self.compressor(output[:, -1, :])))
        return output

    '''
    def extract_feature(self, x):
        # 入力値xから特徴量抽出までの流れを定義する。
        output, _ = self.cell(x)
        output = self.compressor(output[:, -1, :])
        return output
    '''

In [9]:
class FeatureCombinerHandler:
    def __init__(self, feature_combiner_params, store_dir):
        # モデル学習及び推論に用いるデバイスを定義する
        if torch.cuda.device_count() >= 1:
            self.device = 'cuda'
            print("[+] Set Device: GPU")
        else:
            self.device = 'cpu'
            print("[+] Set Device: CPU")

        # モデルのcheckpointや抽出した特徴量及びsentimentをstoreする場所を定義する。
        self.store_dir = store_dir
        os.makedirs(store_dir, exist_ok=True)

        # 上記で作成したfeaturecombinerを定義する。
        self.feature_combiner = FeatureCombiner(**feature_combiner_params).to(
            self.device
        )

        # 学習に用いるoptimizerを定義する。
        self.optimizer = torch.optim.Adam(
            params=self.feature_combiner.parameters(), lr=0.001,
        )

        # ロス関数の定義
        self.criterion = nn.BCELoss().to(self.device)

        # モデルのcheck pointが存在する場合、モデルをロードする
        self._load_model()

    # 学習に必要なデータ(並列のためbatch化されたもの)をサンプルする。
    def _sample_xy(self, data_type):
        assert data_type in ("train", "val")

        # data_typeより、data_typeに合致したデータを取得するようにしている。
        if data_type == "train":
            # dataloaderをiteratorとして定義し、next関数として毎時のデータをサンプルすることができる。
            # Iteratorは全てのデータがサンプルされると、StopIterationのエラーを発するが、そのようなエラーが出たとき、
            # Iteratorを再定義し、データをサンプルするようにしている。
            try:
                x, y = next(self.iterable_train_dataloader)
            except StopIteration:
                self.iterable_train_dataloader = iter(self.train_dataloader)
                x, y = next(self.iterable_train_dataloader)

        elif data_type == "val":
            try:
                x, y = next(self.iterable_val_dataloader)
            except StopIteration:
                self.iterable_val_dataloader = iter(self.val_dataloader)
                x, y = next(self.iterable_val_dataloader)

        return x.to(self.device), y.to(self.device)

    # モデルのパラメータをアップデートするロジック
    def _update_params(self, loss):
        # ロスから、gradientを逆伝播し、パラメータをアップデートする
        loss.backward()
        self.optimizer.step()

    # 学習されたfeature_combinerのパラメータをcheck_pointとしてstoreするロジック
    def _save_model(self, epoch):
        torch.save(
            self.feature_combiner.state_dict(),
            os.path.join(self.store_dir, f"{epoch}.ckpt"),
        )
        print(f"[+] Epoch: {epoch}, Model is saved.")

    # 学習されたcheckpointが存在す場合、feature_combinerにそのパラメータをロードするロジック
    def _load_model(self):
        # cudaで学習されたモデルなどを、cpu環境下でロードするときはこのパラメータが必要となる。
        params_to_load = {}
        if self.device == "cpu":
            params_to_load["map_location"] = torch.device("cpu")

        # .ckptファイルを探し、古い順から新しい順にソートする。
        check_points = glob(os.path.join(self.store_dir, "*.ckpt"))
        check_points = sorted(
            check_points, key=lambda x: int(x.split("/")[-1].replace(".ckpt", "")),
        )

        # check_pointが存在しない場合は、スキップする。
        if len(check_points) == 0:
            print("[!] No exists checkpoint")
            return

        # 複数個のchieck_pointが存在する場合、一番最新のものを使い、モデルのパラメータをロードする
        check_point = check_points[-1]
        self.feature_combiner.load_state_dict(torch.load(check_point, **params_to_load))
        print("[+] Model is loaded")

    # Datasetからdataloaderを定義するロジック
    def _build_dataloader(
        self, dataloader_params, weekly_features, weekly_labels, max_sequence_length
    ):
        # 上記3で作成したしたdatasetを定義する
        dataset = Dataset(
            weekly_features=weekly_features,
            weekly_labels=weekly_labels,
            max_sequence_length=max_sequence_length,
        )

        # datasetのdataをiterableにロードできるよう、dataloaderを定義する、このとき、shuffle=Trueを渡すことで、データはランダムにサンプルされるようになる。
        return DataLoader(dataset=dataset, shuffle=True, **dataloader_params)

    # train用に、featuresとlabelsを渡し、datasetを定義し、dataloaderを定義するロジック
    def set_train_dataloader(
        self, dataloader_params, weekly_features, weekly_labels, max_sequence_length
    ):
        self.train_dataloader = self._build_dataloader(
            dataloader_params=dataloader_params,
            weekly_features=weekly_features,
            weekly_labels=weekly_labels,
            max_sequence_length=max_sequence_length,
        )

        # dataloaderからiteratorを定義する
        # iteratorはnext関数よりデータをサンプルすることが可能となる。
        self.iterable_train_dataloader = iter(self.train_dataloader)

    # validation用に、featuresとlabelsを渡し、datasetを定義し、dataloaderを定義するロジック
    def set_val_dataloader(
        self, dataloader_params, weekly_features, weekly_labels, max_sequence_length
    ):
        self.val_dataloader = self._build_dataloader(
            dataloader_params=dataloader_params,
            weekly_features=weekly_features,
            weekly_labels=weekly_labels,
            max_sequence_length=max_sequence_length,
        )

        # dataloaderからiteratorを定義する
        # iteratorはnext関数よりデータをサンプルすることが可能となる。
        self.iterable_val_dataloader = iter(self.val_dataloader)

    # 学習ロジック
    def train(self, n_epoch):
        # n_epochの回数分、全学習データを複数回用いて学習する。
        for epoch in range(n_epoch):

            # 各々のepochごとのaverage lossを表示するため、lossをstoreするリストを定義する。
            train_losses = []
            test_losses = []

            # train_dataloaderの長さは、全ての学習データを一度用いるときの長さと同様である。
            # batchを組むと、その分train_dataloaderの長さは可変し、ちょうど一度全てのデータで学習できる長さを返す。
            for iter_ in tqdm(range(len(self.train_dataloader))):
                # パラメータをtrainableにするため、feature_combinerをtrainモードにする。
                self.feature_combiner.train()

                # trainデータをサンプルする。
                x, y = self._sample_xy(data_type="train")

                # feature_combinerに特徴量を入力し、sentiment scoreを取得する。
                preds = self.feature_combiner(x=x)

                # sentiment scoreとラベルとのロスを計算する。
                train_loss = self.criterion(preds, y)

                # 計算されたロスは、後ほどepochごとのdisplayに使用するため、storeしておく。
                train_losses.append(train_loss.detach().cpu())

                # lossから、gradientを逆伝播させ、パラメータをupdateする。
                self._update_params(loss=train_loss)

                # validation用のロースを計算する。
                # 毎回計算を行うとコストがかかってくるので、iter_毎5回ごとに計算を行う。
                if iter_ % 5 == 0:

                    # 学習を行わないため、feature_combinerをevalモードにしておく。
                    # evalモードでは、dropoutの影響を受けない。
                    self.feature_combiner.eval()

                    # 各パラメータごとのgradientを計算するとリソースが高まる。
                    # evaluationの時には、gradient情報を持たせないことで、メモリーの節約に繋がる。
                    with torch.no_grad():
                        # validationデータをサンプルする
                        x, y = self._sample_xy(data_type="val")

                        # feature_combinerに特徴量を入力し、sentiment scoreを取得する。
                        preds = self.feature_combiner(x=x)

                        # sentiment scoreとラベルとのロスを計算する。
                        test_loss = self.criterion(preds, y)

                        # 計算されたロスは、後ほどepochごとのdisplayに使用するため、storeしておく。
                        test_losses.append(test_loss.detach().cpu())

            # 毎epoch終了後、平均のロスをプリントする。
            print(
                f"epoch: {epoch}, train_loss: {np.mean(train_losses):.4f}, val_loss: {np.mean(test_losses):.4f}"
            )

            # 毎epoch終了後、モデルのパラメータをstoreする。
            self._save_model(epoch=epoch)

    # 特徴量から、合成特徴量を抽出するロジック
    def combine_features(self, features):
        # 学習を行わないため、feature_combinerをevalモードにしておく。
        self.feature_combiner.eval()

        # gradient情報を持たせないことで、メモリーの節約する。
        with torch.no_grad():

            # 特徴量をfeature_combinerのextract_feature関数に入力し、出力層手前の特徴量を抽出する。
            # 抽出するとき、tensorをcpu上に落とし、np.ndarray形式に変換する。
            return (
                self.feature_combiner.extract_feature(
                    x=torch.tensor(features, dtype=torch.float).to(self.device)
                )
                .cpu()
                .numpy()
            )

    # 特徴量から、翌週のsentimentを予測するロジック
    def predict_sentiment(self, features):
        # 学習を行わないため、feature_combinerをevalモードにしておく。
        self.feature_combiner.eval()

        # gradient情報を持たせないことで、メモリーの節約する。
        with torch.no_grad():

            # 特徴量をfeature_combinerに入力し、sentiment scoreを抽出する。
            # 抽出するとき、tensorをcpu上に落とし、np.ndarray形式に変換する。
            return (
                self.feature_combiner(x=torch.tensor(features, dtype=torch.float).to(self.device))
                .cpu()
                .numpy()
            )

    # weeklyグループされた特徴量を入力に、合成特徴量もしくは、sentiment scoreを抽出するロジック
    def generate_by_weekly_features(
        self, weekly_features, generate_target, max_sequence_length
    ):
        assert generate_target in ("features", "sentiment")
        generate_func = getattr(
            self,
            {"features": "combine_features", "sentiment": "predict_sentiment"}[
                generate_target
            ],
        )

        # グループごとに特徴量もしくは、sentiment scoreを抽出し、最終的に重ねて返すため、リストを作成する。
        outputs = []

        # ユニークな週indexを取得する。
        weeks = sorted(weekly_features.index.get_level_values(0).unique())

        for week in tqdm(weeks):
            # 各週ごとの特徴量を取得し、直近から、max_sequence_length分切る。
            features = weekly_features.xs(week, axis=0, level=0)[-max_sequence_length:]

            # 特徴量をモデルに入力し、合成特徴量もしくは、sentiment scoreを抽出し、outputsにappendする。
            # np.expand_dims(features, axis=0)を用いる理由は、特徴量合成機の入力期待値は、dimention0がbatchであるが、
            # featuresは、[1000, 768]の次元をもち、これらをunsqueezeし、[1, 1000, 768]に変換する必要がある。
            outputs.append(generate_func(features=np.expand_dims(features, axis=0)))

        # outputsを重ね、indexの情報とともにpd.DataFrame形式として返す。
        return pd.DataFrame(np.concatenate(outputs, axis=0), index=weeks)

## 特徴量合成モデルの学習及び特徴量合成

In [10]:
boundary_week = (2020, 53)
features = headline_features
weekly_features = build_weekly_features(features, boundary_week)

<ipython-input-6-94966c175c5c>:3: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  return pd.Series(list(zip(df.index.year, df.index.week)), index=df.index)


In [11]:
# 各セクターのラベルを結合する
weekly_labels = build_weekly_labels(dict_stock_price[0], boundary_week)
weekly_labels_train = weekly_labels['train'].copy()
weekly_labels_test = weekly_labels['test'].copy()

for i in range(1, 18):
    weekly_labels = build_weekly_labels(dict_stock_price[i], boundary_week)
    weekly_labels_train = pd.concat([weekly_labels_train, weekly_labels['train']], axis=1)
    weekly_labels_test = pd.concat([weekly_labels_test, weekly_labels['test']], axis=1)

weekly_labels_train.columns = [i for i in range(18)]
weekly_labels_test.columns = [i for i in range(18)]
weekly_labels_train.index = pd.MultiIndex.from_tuples(weekly_labels_train.index)
weekly_labels_test.index = pd.MultiIndex.from_tuples(weekly_labels_test.index)

display(weekly_labels_train.head(3))
display(weekly_labels_test.head(3))

weekly_labels.clear()
weekly_labels['train'] = weekly_labels_train
weekly_labels['test'] = weekly_labels_test

<ipython-input-6-94966c175c5c>:3: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  return pd.Series(list(zip(df.index.year, df.index.week)), index=df.index)
<ipython-input-6-94966c175c5c>:3: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  return pd.Series(list(zip(df.index.year, df.index.week)), index=df.index)
<ipython-input-6-94966c175c5c>:3: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you m

0    1    2    3    4    5    6    7    8    9    10   11   12   13  \
2020 2  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
     3  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0   
     4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   

         14   15   16   17  
2020 2  0.0  0.0  0.0  0.0  
     3  0.0  0.0  1.0  1.0  
     4  0.0  1.0  1.0  0.0

0    1    2    3    4    5    6    7    8    9    10   11   12   13  \
2021 1  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0   
     2  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0   
     3  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

         14   15   16   17  
2021 1  0.0  0.0  0.0  0.0  
     2  1.0  0.0  0.0  1.0  
     3  0.0  0.0  0.0  0.0

In [12]:
feature_combiner_handler = FeatureCombinerHandler(feature_combiner_params={"input_size": 768, "hidden_size": 128}, store_dir='./test')

[+] Set Device: CPU
[!] No exists checkpoint


In [13]:
# train dataloaderをsetする。
feature_combiner_handler.set_train_dataloader(
    dataloader_params={
        "batch_size": 4,
        "num_workers": 2,
    },
    weekly_features=weekly_features['train'],
    weekly_labels=weekly_labels['train'],
    max_sequence_length=1000
)

# validation dataloaderをsetする。
feature_combiner_handler.set_val_dataloader(
    dataloader_params={
        "batch_size": 4,
        "num_workers": 2,
    },
    weekly_features=weekly_features['test'],
    weekly_labels=weekly_labels['test'],
    max_sequence_length=1000
)

In [16]:
# Check
tmp = iter(feature_combiner_handler.train_dataloader)
y = tmp.next()[1]
pred = feature_combiner_handler.feature_combiner(tmp.next()[0])
display(y)
display(pred)

loss = nn.BCELoss()
loss(y.detach(), pred.detach())

tensor([[1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

tensor([[0.4956, 0.5101, 0.4692, 0.4938, 0.5008, 0.5303, 0.4837, 0.5153, 0.4873,
         0.5172, 0.5067, 0.5068, 0.4868, 0.5066, 0.4784, 0.5138, 0.5002, 0.4945],
        [0.4981, 0.5100, 0.4752, 0.5052, 0.4970, 0.5311, 0.4868, 0.5112, 0.4910,
         0.5061, 0.5029, 0.5191, 0.4871, 0.5036, 0.4877, 0.5238, 0.4999, 0.4979],
        [0.5063, 0.5126, 0.4760, 0.5137, 0.5057, 0.5349, 0.4781, 0.5056, 0.4974,
         0.5227, 0.4993, 0.5007, 0.4836, 0.5183, 0.4879, 0.5162, 0.5002, 0.4907],
        [0.4961, 0.5051, 0.4834, 0.4965, 0.4965, 0.5317, 0.4825, 0.5090, 0.4943,
         0.5255, 0.4979, 0.5203, 0.4861, 0.5030, 0.4796, 0.5100, 0.5063, 0.4970]],
       grad_fn=<SigmoidBackward>)

tensor(49.9232)

マルチターゲットにおける損失関数について  
https://towardsdatascience.com/multi-label-image-classification-with-neural-network-keras-ddc1ab1afede

In [17]:
feature_combiner_handler.train(n_epoch=20)

tensor([[0.5022, 0.5008, 0.4743, 0.4992, 0.5079, 0.5236, 0.4812, 0.5120, 0.4925,
         0.5125, 0.4995, 0.5170, 0.4876, 0.5154, 0.4789, 0.5178, 0.4996, 0.4907],
        [0.5072, 0.5095, 0.4765, 0.5007, 0.4962, 0.5243, 0.4927, 0.5305, 0.4852,
         0.5164, 0.5002, 0.5076, 0.4837, 0.5136, 0.4980, 0.5163, 0.5067, 0.4980],
        [0.5040, 0.5053, 0.4681, 0.4996, 0.5077, 0.5324, 0.4773, 0.5110, 0.4914,
         0.5193, 0.5004, 0.5141, 0.4884, 0.5216, 0.4898, 0.5180, 0.5076, 0.4855],
        [0.5069, 0.5018, 0.4684, 0.4967, 0.5028, 0.5267, 0.4879, 0.5079, 0.4938,
         0.5187, 0.5076, 0.5182, 0.4922, 0.5175, 0.4976, 0.5291, 0.5101, 0.5003]],
       grad_fn=<SigmoidBackward>) tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1.],
        [1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.

KeyboardInterrupt: 

つづきはGoogleColaboratoryで行う